- voting : 같은 데이터셋, 다른 분류기 // 2가지 종류
- bagging : 다른 데이터셋, 같은 분류기
- boosting :

# VotingRegression (보팅 회귀)
- 여러 회귀모형 알고리즘을 결합하고 그것의 평균 예측 값을 반환

## 패키지 로딩

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import Lasso, Ridge, LinearRegression
# Lasso : L1 규제가 적용된 회귀 모형 # Ridge : L2 규제가 적용된 회귀 모형
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import numpy as np
import pandas as pd

## 데이터 로딩 및 분할

In [7]:
x, y = load_boston(return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    train_size=0.7, 
                                                    random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((354, 13), (152, 13), (354,), (152,))

## 모델 생성 및 평가

In [9]:
# 세 가지의 회귀 모형 모두 생성하기
lasso = Lasso(alpha=0.5) # 중요하지 않은 가중치를 0으로하여 삭제
ridge = Ridge(alpha=1)
linear = LinearRegression()

# 보팅 모델 생성
vo_r = VotingRegressor(estimators=[('Lasso', lasso), ('Ridge', ridge), ('Linear', linear)])
# 튜플의 형태로 전달

# 모델 학습
vo_r.fit(x_train, y_train)

# 모델 검증
r_square = vo_r.score(x_test, y_test)
print(f'결정계수: {r_square:.3f}')

# 예측
y_hat = vo_r.predict(x_test)

# 모델 평가
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'RMSE: {rmse:.3f}')

결정계수: 0.666
RMSE: 5.274


# VotingClassifier (보팅 분류)

## 패키지 로딩

In [11]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score # 분류 모델 성능 측정용 평가지표

import numpy as np
import pandas as pd

## 데이터 로딩 및 분할

In [12]:
x, y = load_breast_cancer(return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    train_size=0.7, 
                                                    random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((398, 30), (171, 30), (398,), (171,))

## 모델 생성 및 검증

In [18]:
# 두 가지의 분류 모형 모두 생성하기
logistic = LogisticRegression(max_iter=3000)
knn = KNeighborsClassifier()

# 보팅 모델 생성
vo_c = VotingClassifier(estimators=[('Logistic', logistic), ('KNN', knn)], voting='soft')
# voting : 하드보팅은 'hard', 소프트보팅은 'soft' 값 전달 (DEFAULT는 hard)
# VotingRegression은 회귀분석이라 분류기들의 평균 예측값을 반환하므로 voting 지정 불가
# VotingClasssifier은 분류분석이라 0 또는 1을 반환하므로 투표 방식을 지정 가능


# 모델 학습
vo_c.fit(x_train, y_train)

# 예측
y_hat = vo_c.predict(x_test)

# 모델 평가
print(f'정확도: {accuracy_score(y_test, y_hat):.3f}')
print(f'AUC: {roc_auc_score(y_test, vo_c.predict_proba(x_test)[:,1]):.3f}')

정확도: 0.965
AUC:0.994


# GradientBoostingClassifier (부스팅 분류)

## 패키지 로딩

In [19]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score # 분류 모델 성능 측정용 평가지표

import numpy as np
import pandas as pd

## 데이터 로딩 및 분할

In [20]:
x, y = load_breast_cancer(return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    train_size=0.7, 
                                                    random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((398, 30), (171, 30), (398,), (171,))

## 모델 생성 및 평가
- GradientBoostingClassifier
     - 랜덤 포레스트와 같이 의사결정나무 모델을 부스팅 방법으로 활용하는 모델
     - 이전 예측기가 만든 잔여오차(residual error)에 새로운 예측기를 학습시킴

In [23]:
# 모델 생성
gb_c = GradientBoostingClassifier()

# 모델 학습
gb_c.fit(x_train, y_train)

# 예측
y_hat = gb_c.predict(x_test)

# 모델 평가
print(f'정확도: {accuracy_score(y_test, y_hat):.3f}')
print(f'AUC: {roc_auc_score(y_test, gb_c.predict_proba(x_test)[:,1]):.3f}')

정확도: 0.965
AUC: 0.997


## 하이퍼 파라미터 튜닝

In [27]:
from sklearn.model_selection import GridSearchCV

gbc_params = {'n_estimators': [100,200],
              'max_depth': [6,8,10,12], # 나무의 깊이
              'min_samples_leaf' : [3,5,7,19], # 잎노드의 최소 샘플 수
              'min_samples_split' : [3,5,7,10] # 잎노드 분할시 최소 샘플 수
              }
# gb_c = GradientBoostingClassifier()
gbc_gs = GridSearchCV(gb_c, gbc_params, scoring='accuracy', n_jobs=-1)
gbc_gs.fit(x_train, y_train)

GridSearchCV(estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [3, 5, 7, 19],
                         'min_samples_split': [3, 5, 7, 10],
                         'n_estimators': [100, 200]},
             scoring='accuracy')

In [28]:
print('최적 파라미터:', gbc_gs.best_params_)
# best_params_ : 최적의 파라미터 조합 결과 반환
print('예측 정확도:', gbc_gs.best_score_)

최적 파라미터: {'max_depth': 12, 'min_samples_leaf': 7, 'min_samples_split': 10, 'n_estimators': 200}
예측 정확도: 0.9623417721518986


In [ ]:
# r2_score : 현실에서는 0.5를 넘기가 어렵기 때문에, 0.3, 0.4만 나와도 유용하다고 간주한다. 모호하기 때문!
# ex. 행복 만족도 조사시 여러가지 독립변수 입력값의 개인차가 심하므로

# GradientBoostingRegressor (부스팅 회귀)

## 패키지 로딩

In [29]:
from sklearn.datasets import load_diabetes
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd

## 데이터 로딩 및 분할

In [30]:
x, y = load_diabetes(return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    train_size=0.7, 
                                                    random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((309, 10), (133, 10), (309,), (133,))

## 모델 생성 및 평가

In [32]:
# 모델 생성
gb_r = GradientBoostingRegressor(n_estimators=120, max_depth=6)

# 모델 학습
gb_r.fit(x_train, y_train)

GradientBoostingRegressor(max_depth=6, n_estimators=120)

In [34]:
# 예측
y_hat = gb_r.predict(x_test)

# 모델 평가
r_square = gb_r.score(x_test, y_test)
print(f'결정계수: {r_square:.3f}')
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print(f'RMSE: {rmse:.3f}')

결정계수: 0.216
RMSE: 63.250
